In [58]:
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder

In [60]:
#creating data part

In [62]:
images=[]
globalpath='C:\\Users\\jerar\\OneDrive\\Desktop\\food_classify\\images\\'
os.chdir(globalpath)
print(os.getcwd())
labels=os.listdir()#list of all foodnames since food name is the directory name


C:\Users\jerar\OneDrive\Desktop\food_classify\images


In [64]:
### creating a data frame consisiting of following fields imagepath,label
###

In [66]:
data={"image":[],
    "label":[]
}
for label in labels:
    os.chdir(globalpath)
    path=os.getcwd()+"\\"+label
    os.chdir(path)
    imagelist=os.listdir()
    labellist=[]
    for i in range(0,len(imagelist)):
        imagelist[i]=path+"\\"+imagelist[i]
        labellist.append(label)
    data["image"].extend(imagelist)
    data["label"].extend(labellist)


'''
data["label"]=list(encoded_labels)'''
#load data into a DataFrame object:
df = pd.DataFrame(data)
df.head()


,image,label
0,C:\Users\jerar\OneDrive\Desktop\food_classify\...,Biryani
1,C:\Users\jerar\OneDrive\Desktop\food_classify\...,Biryani
2,C:\Users\jerar\OneDrive\Desktop\food_classify\...,Biryani
3,C:\Users\jerar\OneDrive\Desktop\food_classify\...,Biryani
4,C:\Users\jerar\OneDrive\Desktop\food_classify\...,Biryani


In [76]:
#encoding labels
encoder=LabelEncoder()
encoded_labels= encoder.fit_transform(df.iloc[:,1])
df.iloc[:,1]=encoded_labels
print(df.head())

                                               image label
0  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0
1  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0
2  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0
3  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0
4  C:\Users\jerar\OneDrive\Desktop\food_classify\...     0


In [70]:
### creating tensors from dictionary using tf.data.Dataset.from_tensor_slices(tensors)
### tensors should be 1D arrays
###

In [140]:
images=df.iloc[:,0:1].values #using 0:1 since even though there is single column , X should be 2d and y should be 1D
labels=df.iloc[:,1].values
X_train, X_test, y_train, y_test=train_test_split(images, labels, test_size=0.2)

In [142]:
def preprocess(imagepath, label):
    file = tf.io.read_file(imagepath)  # Read image file
    image = tf.image.decode_jpeg(file, channels=3)  # Decode JPEG file into an image
    image = tf.image.resize(image, [128, 128])  # Resize to 128x128
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

In [168]:
train_data=tf.data.Dataset.from_tensor_slices((list(X_train[:,0]), list(y_train))).map(preprocess)
test_data= tf.data.Dataset.from_tensor_slices((list(X_test[:,0]), list(y_test))).map(preprocess)

In [158]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # for 2-class classification
])


In [160]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  #  'categorical_crossentropy' if labels are one-hot encoded
    metrics=['accuracy']
)
batch_size = 32
train_dataset = train_data.batch(batch_size)
model.fit(train_dataset, epochs=7)

Epoch 1/7
14/14 [==============================] - 4s 179ms/step - loss: 1.1155 - accuracy: 0.5608
Epoch 2/7
14/14 [==============================] - 2s 150ms/step - loss: 0.5399 - accuracy: 0.7703
Epoch 3/7
14/14 [==============================] - 2s 151ms/step - loss: 0.3499 - accuracy: 0.8514
Epoch 4/7
14/14 [==============================] - 2s 138ms/step - loss: 0.2641 - accuracy: 0.8716
Epoch 5/7
14/14 [==============================] - 2s 140ms/step - loss: 0.1531 - accuracy: 0.9392
Epoch 6/7
14/14 [==============================] - 2s 134ms/step - loss: 0.1996 - accuracy: 0.9099
Epoch 7/7
14/14 [==============================] - 2s 130ms/step - loss: 0.2872 - accuracy: 0.8851


In [ ]:
print(type(train_data)) # should be (num_samples, 128, 128, 3)


In [170]:
test_data = test_data.batch(batch_size)
loss, accuracy = model.evaluate(test_data)
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")

4/4 [==============================] - 1s 60ms/step - loss: 0.5003 - accuracy: 0.7658
Test loss: 0.5002889633178711
Test accuracy: 0.7657657861709595
